# DSCI 100 Group 177 - Group Project Proposal

### Title: [placeholder]

### Introduction:

[placeholder]

### Preliminary exploratory data analysis:

[placeholder]

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [4]:
pubg_weapons <- read_csv("data/pubg-weapon-stats.csv")
colnames(pubg_weapons) <- make.names(colnames(pubg_weapons))
pubg_weapons <- mutate(pubg_weapons, Weapon.Type = as_factor(Weapon.Type))

pubg_weapons

Rows: 44 Columns: 20
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): Weapon Name, Weapon Type, Fire Mode
dbl (17): Bullet Type, Damage, Magazine Capacity, Range, Bullet Speed, Rate ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Weapon.Name,Weapon.Type,Bullet.Type,Damage,Magazine.Capacity,Range,Bullet.Speed,Rate.of.Fire,Shots.to.Kill..Chest.,Shots.to.Kill..Head.,Damage.Per.Second,Fire.Mode,BDMG_0,BDMG_1,BDMG_2,BDMG_3,HDMG_0,HDMG_1,HDMG_2,HDMG_3
<chr>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Groza,Assault Rifle,7.62,49,30,400,715,0.0800,4,2,612.00,"Single, Automatic, Burst",47.0,34.3,29.4,22.0,115.1,80.6,69.0,51.8
AKM,Assault Rifle,7.62,49,30,400,710,0.0100,4,2,490.00,"Single, Automatic",47.0,34.3,29.4,22.0,115.1,80.6,69.0,51.8
M762,Assault Rifle,7.62,47,30,400,715,0.0860,4,2,547.00,"Single, Automatic, Burst",46.0,32.9,28.2,21.1,110.4,77.3,66.2,49.7
MK47 Mutant,Assault Rifle,7.62,49,30,500,715,0.0100,4,2,490.00,"Single, Burst",49.0,34.3,29.4,22.0,115.1,80.6,69.0,51.8
AUG A3,Assault Rifle,5.56,43,30,600,880,0.0860,4,2,502.00,"Single, Automatic",41.0,30.1,25.8,19.3,101.0,70.7,60.6,45.4
SCAR-L,Assault Rifle,5.56,43,30,600,880,0.0960,4,2,448.00,"Single, Automatic",41.0,30.1,25.8,19.3,101.0,70.7,60.6,45.4
M416,Assault Rifle,5.56,43,30,600,890,0.0860,4,2,502.00,"Single, Automatic",41.0,30.1,25.8,19.3,101.0,70.7,60.6,45.4
M16A4,Assault Rifle,5.56,43,30,600,900,0.0100,4,2,430.00,"Single, Burst",43.0,30.1,25.8,19.3,101.0,70.7,60.6,45.4
G36C,Assault Rifle,5.56,43,30,600,880,0.0860,4,2,502.00,"Single, Automatic",41.0,30.1,25.8,19.3,101.0,70.7,60.6,45.4


### Methods:

[placeholder]

### Expected outcome and significance:

[placeholder]